# ETL USANDO APACHE SPARK

## Extract

#### Instalación de apache spark

In [6]:
pip install pyspark

#### Importación de librerías que iré ocupando y creación de la sesión de Spark y agrego el UTF 8 para el tema de los acentos

In [7]:
from google.colab import files
from pyspark.sql import SparkSession
from pyspark.sql.functions import monotonically_increasing_id,split,explode,expr,regexp_extract,trim,regexp_replace,col, to_date, concat, substring

# Sesión de sparl
spark = SparkSession.builder \
    .appName("ETL FIFA mundial") \
    .config("spark.sql.parquet.outputEncoding", "UTF-8") \
    .getOrCreate()

####  Leémos el CSV suvbido

In [8]:
# Damos la ruta del mi archivo CSV
csv_file_path = "/content/dataset.csv"

# Leo y paso el csv a utf-8 para poder ver los caracteres de manera correcta
df = spark.read.csv(csv_file_path, header=True, inferSchema=True, encoding="UTF-8")

# Veamos las primeras 5 filas y su esquema
print("Schema:")
df.printSchema()
print("Primeras 5 filas del DataFrame:")
df.show(5, truncate=False)

Schema:
root
 |-- home_team: string (nullable = true)
 |-- away_team: string (nullable = true)
 |-- away_penalty: integer (nullable = true)
 |-- Attendance: integer (nullable = true)
 |-- Venue: string (nullable = true)
 |-- Officials: string (nullable = true)
 |-- Round: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Referee: string (nullable = true)
 |-- Notes: string (nullable = true)
 |-- Host: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- home_goal: string (nullable = true)
 |-- away_goal: string (nullable = true)
 |-- home_penalty_goal: string (nullable = true)
 |-- away_penalty_goal: string (nullable = true)
 |-- home_red_card: string (nullable = true)
 |-- away_red_card: string (nullable = true)
 |-- home_yellow_red_card: string (nullable = true)
 |-- away_yellow_red_card: string (nullable = true)

Primeras 5 filas del DataFrame:
+---------+---------+------------+----------+-----------------------------------+-----------------------------

#### Empezaré a eliminar todas las columnas que sé que no me interesan analisar, muchas de ellas tienen la palbra long por lo que me ayudaré de un ciclo for y un extend

In [9]:
# Lista de columnas a eliminar
delete = [
    "home_xg","away_xg","home_manager", "away_manager", "home_manager", "away_manager","home_captain","away_penalty","Score",
    "away_captain", "home_captain", "away_captain","Venue", "Officials","Notes","home_score","away_score","home_penalty",
    "home_own_goal", "away_own_goal","home_penalty_miss_long","away_penalty_miss_long","home_yellow_red_card","away_yellow_red_card",
]

# Tampoco necesito las columnas long menos las de penales
long_quit = [col for col in df.columns if "long" in col and col not in ["home_penalty_miss_long", "away_penalty_miss_long"]]
delete.extend(long_quit)

# Elimino del df
df_clean = df.drop(*delete)

# Así me quedó mi df
print("Columnas después de la limpieza:")
df_clean.printSchema()

Columnas después de la limpieza:
root
 |-- home_team: string (nullable = true)
 |-- away_team: string (nullable = true)
 |-- Attendance: integer (nullable = true)
 |-- Round: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Referee: string (nullable = true)
 |-- Host: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- home_goal: string (nullable = true)
 |-- away_goal: string (nullable = true)
 |-- home_penalty_goal: string (nullable = true)
 |-- away_penalty_goal: string (nullable = true)
 |-- home_red_card: string (nullable = true)
 |-- away_red_card: string (nullable = true)



#### He visto que es buen standard manejar los archivos en parquet por varias ventajas entonces aquí haré lo mismo

In [10]:
#Especifico a dónde se va ir el parquet
parquet_file = "/content/world_cup.parquet"

# Guardo en parquet
df_clean.write.parquet(parquet_file)

# Leo el archivo y lo muestro
df_parquet = spark.read.parquet(parquet_file, encoding="UTF-8")
df_parquet.show(5)
df_parquet.printSchema()

+---------+---------+----------+-----------------+----------+--------------------+-----+----+--------------------+------------------+--------------------+--------------------+-------------+-------------+
|home_team|away_team|Attendance|            Round|      Date|             Referee| Host|Year|           home_goal|         away_goal|   home_penalty_goal|   away_penalty_goal|home_red_card|away_red_card|
+---------+---------+----------+-----------------+----------+--------------------+-----+----+--------------------+------------------+--------------------+--------------------+-------------+-------------+
|Argentina|   France|     88966|            Final|18/12/2022|    Szymon Marciniak|Qatar|2022|Ángel Di María · ...|Kylian Mbappé · 81|Lionel Messi (P) ...|Kylian Mbappé (P)...|         NULL|         NULL|
|  Croatia|  Morocco|     44137|Third-place match|17/12/2022|Abdulrahman Ibrah...|Qatar|2022|Joško Gvardiol · ...|   Achraf Dari · 9|                NULL|                NULL|         

## Transform

### Dim tables

#### Dimensión de los equipos extrayendo de las 2 columnas donde se puedan extraer

In [11]:
# Extraemos los equipos de home_team y away_team
dim_team = df_parquet.select("home_team").distinct().withColumnRenamed("home_team", "team")
dim_team2 = df_parquet.select("away_team").distinct().withColumnRenamed("away_team", "team")

# Unimos y quitamos los que se repiten porque los países pueden l o v
dim_teams = dim_team.union(dim_team2).distinct()
# Asignamos un ID único
dim_teams = dim_teams.withColumn("id", monotonically_increasing_id() + 1)

# Así quedaría nuestra dimensión de países
dim_teams.show(5)

+-------------+---+
|         team| id|
+-------------+---+
|Côte d'Ivoire|  1|
|       Russia|  2|
|     Paraguay|  3|
|      Senegal|  4|
|       Sweden|  5|
+-------------+---+
only showing top 5 rows



#### Dimensión árbitros donde solo extrameos la columna quitamos los duplicados y asignamos ID

In [12]:
# Extraer los árbitros de la columna 'Referee' y eliminar duplicados
dim_referees = df_parquet.select("Referee").distinct()

# Asignar un ID
dim_referees = dim_referees.withColumn("id", monotonically_increasing_id() + 1)

# Así quedaría nuestra dimensión de árbitros
dim_referees.show(5)

+--------------------+---+
|             Referee| id|
+--------------------+---+
|    Mustapha Ghorbal|  1|
|  Roberto Goicoechea|  2|
|      Leslie Mottram|  3|
|       Anibal Tejada|  4|
|Arturo Brizio Carter|  5|
+--------------------+---+
only showing top 5 rows



#### Hacemos la dimensión de partidos donde extraemos los datos relevantes y establecemos la fecha a dim date

In [13]:
# Pasaré lo de las fechas a un formato de dim date
dim_matches = df_parquet.withColumn("Date",
    concat(
        substring(col("Date"), 7, 4),  # Año
        substring(col("Date"), 4, 2),  # Mes
        substring(col("Date"), 1, 2)   # Día
    )
)

# Seleccionar las columnas para hacer mi dimesioón de los partidos
dim_matches = dim_matches.select(
    col("Attendance"),
    col("Round"),
    col("Host"),
    col("Date")
)

# Asigno el ID
dim_matches = dim_matches.withColumn("id", monotonically_increasing_id() + 1)

# Así quedaría nuestra dimensión de partidos
dim_matches.show(5)



+----------+-----------------+-----+--------+---+
|Attendance|            Round| Host|    Date| id|
+----------+-----------------+-----+--------+---+
|     88966|            Final|Qatar|20221218|  1|
|     44137|Third-place match|Qatar|20221217|  2|
|     68294|      Semi-finals|Qatar|20221214|  3|
|     88966|      Semi-finals|Qatar|20221213|  4|
|     44198|   Quarter-finals|Qatar|20221210|  5|
+----------+-----------------+-----+--------+---+
only showing top 5 rows



#### En la dimensión de jugadores sacamos las columnas donde podamos extraer jugadores de los eventos a estudiar y los tratamos a manera de poder extraerlos

In [14]:
# Selecciono las columnas donde tienen jugadores
dim_players = (
    df_parquet.selectExpr("home_goal as player", "home_team as team")
    .union(df_parquet.selectExpr("away_goal as player", "away_team as team"))
    .union(df_parquet.selectExpr("home_red_card as player", "home_team as team"))
    .union(df_parquet.selectExpr("away_red_card as player", "away_team as team"))
    .union(df_parquet.selectExpr("home_penalty_goal as player", "home_team as team"))
    .union(df_parquet.selectExpr("away_penalty_goal as player", "away_team as team"))
)

# Empecé a quitar caracteres para poder extraer los jugadores separandolos por,
dim_players = dim_players.withColumn("player", regexp_replace(col("player"), r"\(P\)", ""))
dim_players = dim_players.withColumn("player", regexp_replace(col("player"), "\s+", ""))
dim_players = dim_players.withColumn("player", regexp_replace(col("player"), "·", ""))
dim_players = dim_players.withColumn("player", regexp_replace(col("player"), "[0-9]", ""))
dim_players = dim_players.withColumn("player", regexp_replace(col("player"), "\|", ","))
dim_players = dim_players.withColumn("player", regexp_replace(col("player"), r'(?<=[a-z])(?=[A-Z])', ' '))
dim_players = dim_players.withColumn("player", regexp_replace(col("player"), "\+", ""))
dim_players = dim_players.withColumn("player", trim(col("player")))

# Searo los jugaodres
dim_players = dim_players.withColumn("player", explode(split(col("player"), ",")))
dim_players = dim_players.withColumn("player", trim(col("player")))

# Elimino duplicado
dim_players = dim_players.dropDuplicates(["player"])

# Asigno el ID
dim_players = dim_players.withColumn("id1", monotonically_increasing_id() + 1)

# Uno con dim team para extraer el id y quito la columna team porque ya no la ocupo
dim_players = dim_players.join(dim_teams, dim_players.team == dim_teams.team, "inner")
dim_players = dim_players.drop('team')

# Renombro columnas
dim_players = dim_players.withColumnRenamed('id','team_id')
dim_players = dim_players.withColumnRenamed('id1','id')

# Muestra los primeros 5 registros
dim_players.show(5)

+--------------------+---+-------+
|              player| id|team_id|
+--------------------+---+-------+
|    Abdeljalil Hadda|  1|     40|
|     Abdelkrim Merry|  2|     40|
| Abdelmoumene Djabou|  3|     13|
|   Abderrazak Khairi|  4|     40|
|Abdullah Al-Buloushi|  5|     84|
+--------------------+---+-------+
only showing top 5 rows



### Fact tables

#### Hechos de goles

##### Primero empiezo por los goles del local con la misma lógica de jugadores

In [15]:
# Selecciono los goles local
f_goles1 = df_parquet.select(
    df_parquet['home_goal'].alias('player'),
)

# Asignar un ID que es el del partido
f_goles1 = f_goles1.withColumn("match_id", monotonically_increasing_id() + 1)

# Limpiamos player para extraer fácil el nombre
f_goles1 = f_goles1.withColumn("player", regexp_replace(col("player"), r"\(P\)", ""))
f_goles1 = f_goles1.withColumn("player", regexp_replace(col("player"), "\s+", ""))
f_goles1 = f_goles1.withColumn("player", regexp_replace(col("player"), "·", ""))
f_goles1 = f_goles1.withColumn("player", regexp_replace(col("player"), "\|", ","))
f_goles1 = f_goles1.withColumn("player", regexp_replace(col("player"), r'(?<=[a-z])(?=[A-Z])', ' '))
f_goles1 = f_goles1.withColumn("player", regexp_replace(col("player"), r"\+.+", ""))
f_goles1 = f_goles1.withColumn("player", trim(col("player")))

# Hacemos un registro para cada jugador
f_goles1 = f_goles1.withColumn("player", explode(split(col("player"), ",")))
f_goles1 = f_goles1.withColumn("player", trim(col("player")))

# Quitamos el minuto del jugador y lo pasamos a la columna minute
f_goles1 = f_goles1.withColumn("minute", regexp_extract(col("player"), r'(\d+)$', 1))
f_goles1 = f_goles1.withColumn("player", regexp_extract(col("player"), r'^(.*?)\d+', 1))

# Uno con dim team para extraer el id y quito la columna team porque ya no la ocupo
f_goles1 = f_goles1.join(dim_players, f_goles1.player == dim_players.player, "inner")
f_goles1 = f_goles1.drop('team','player','team_id')
f_goles1 = f_goles1.withColumnRenamed('id','player_id')

# Así quedaría
f_goles1.show(5)

+--------+------+---------+
|match_id|minute|player_id|
+--------+------+---------+
|     448|    59|        1|
|     962|     1|        9|
|     911|    79|       11|
|     911|    30|       11|
|     901|     4|       11|
+--------+------+---------+
only showing top 5 rows



##### Ahora goles de visita

In [16]:
# Selecciono los goles visitantes
f_goles2 = df_parquet.select(
    df_parquet['away_goal'].alias('player'),
)

# Asignar un ID de partido
f_goles2 = f_goles2.withColumn("match_id", monotonically_increasing_id() + 1)

# Limpiamos player para extraer fácil el nombre
f_goles2 = f_goles2.withColumn("player", regexp_replace(col("player"), r"\(P\)", ""))
f_goles2 = f_goles2.withColumn("player", regexp_replace(col("player"), "\s+", ""))
f_goles2 = f_goles2.withColumn("player", regexp_replace(col("player"), "·", ""))
f_goles2 = f_goles2.withColumn("player", regexp_replace(col("player"), "\|", ","))
f_goles2 = f_goles2.withColumn("player", regexp_replace(col("player"), r'(?<=[a-z])(?=[A-Z])', ' '))
f_goles2 = f_goles2.withColumn("player", regexp_replace(col("player"), r"\+.+", ""))
f_goles2 = f_goles2.withColumn("player", trim(col("player")))

# Hacemos un registro para cada jugador
f_goles2 = f_goles2.withColumn("player", explode(split(col("player"), ",")))
f_goles2 = f_goles2.withColumn("player", trim(col("player")))

# Quitamos el minuto del jugador y lo pasamos a la columna minute
f_goles2 = f_goles2.withColumn("minute", regexp_extract(col("player"), r'(\d+)$', 1))
f_goles2 = f_goles2.withColumn("player", regexp_extract(col("player"), r'^(.*?)\d+', 1))

# Uno con dim team para extraer el id y quito la columna team porque ya no la ocupo
f_goles2 = f_goles2.join(dim_players, f_goles2.player == dim_players.player, "inner")
f_goles2 = f_goles2.drop('team','player','team_id')
f_goles2 = f_goles2.withColumnRenamed('id','player_id')

# Así quedaría
f_goles2.show(5)

+--------+------+---------+
|match_id|minute|player_id|
+--------+------+---------+
|     416|    47|        1|
|     576|    62|        2|
|     162|    38|        3|
|     140|   120|        3|
|     576|    27|        4|
+--------+------+---------+
only showing top 5 rows



##### Ahora goles de penal local

In [17]:
# Selecciono los penales
f_goles3 = df_parquet.select(
    df_parquet['home_penalty_goal'].alias('player'),
)

# Asignar un ID del partido
f_goles3 = f_goles3.withColumn("match_id", monotonically_increasing_id() + 1)

# Limpiamos player para extraer fácil el nombre
f_goles3 = f_goles3.withColumn("player", regexp_replace(col("player"), r"\(P\)", ""))
f_goles3 = f_goles3.withColumn("player", regexp_replace(col("player"), "\s+", ""))
f_goles3 = f_goles3.withColumn("player", regexp_replace(col("player"), "·", ""))
f_goles3 = f_goles3.withColumn("player", regexp_replace(col("player"), "\|", ","))
f_goles3 = f_goles3.withColumn("player", regexp_replace(col("player"), r'(?<=[a-z])(?=[A-Z])', ' '))
f_goles3 = f_goles3.withColumn("player", regexp_replace(col("player"), r"\+.+", ""))
f_goles3 = f_goles3.withColumn("player", trim(col("player")))

# Hacemos un registro para cada jugador
f_goles3 = f_goles3.withColumn("player", explode(split(col("player"), ",")))
f_goles3 = f_goles3.withColumn("player", trim(col("player")))

# Quitamos el minuto del jugador y lo pasamos a la columna minute
f_goles3 = f_goles3.withColumn("minute", regexp_extract(col("player"), r'(\d+)$', 1))
f_goles3 = f_goles3.withColumn("player", regexp_extract(col("player"), r'^(.*?)\d+', 1))

# Uno con dim team para extraer el id y quito la columna team porque ya no la ocupo
f_goles3 = f_goles3.join(dim_players, f_goles3.player == dim_players.player, "inner")
f_goles3 = f_goles3.drop('team','player','team_id')
f_goles3 = f_goles3.withColumnRenamed('id','player_id')

# Así quedaría
f_goles3.show(5)

+--------+------+---------+
|match_id|minute|player_id|
+--------+------+---------+
|     800|    84|       14|
|     458|    18|       34|
|     590|     6|       52|
|     744|    50|       69|
|     501|    85|       82|
+--------+------+---------+
only showing top 5 rows



##### Por último hacemos los penales de visita

In [18]:
# Selecciono los goles de penal de visita
f_goles4 = df_parquet.select(
    df_parquet['away_penalty_goal'].alias('player'),
)

# Asignar un ID único a cada registro
f_goles4 = f_goles4.withColumn("match_id", monotonically_increasing_id() + 1)

# Limpiamos player para extraer fácil el nombre
f_goles4 = f_goles4.withColumn("player", regexp_replace(col("player"), r"\(P\)", ""))
f_goles4 = f_goles4.withColumn("player", regexp_replace(col("player"), "\s+", ""))
f_goles4 = f_goles4.withColumn("player", regexp_replace(col("player"), "·", ""))
f_goles4 = f_goles4.withColumn("player", regexp_replace(col("player"), "\|", ","))
f_goles4 = f_goles4.withColumn("player", regexp_replace(col("player"), r'(?<=[a-z])(?=[A-Z])', ' '))
f_goles4 = f_goles4.withColumn("player", regexp_replace(col("player"), r"\+.+", ""))
f_goles4 = f_goles4.withColumn("player", trim(col("player")))

# Hacemos un registro para cada jugador
f_goles4 = f_goles4.withColumn("player", explode(split(col("player"), ",")))
f_goles4 = f_goles4.withColumn("player", trim(col("player")))

# Quitamos el minuto del jugador y lo pasamos a la columna minute
f_goles4 = f_goles4.withColumn("minute", regexp_extract(col("player"), r'(\d+)$', 1))
f_goles4 = f_goles4.withColumn("player", regexp_extract(col("player"), r'^(.*?)\d+', 1))

# Uno con dim team para extraer el id y quito la columna team porque ya no la ocupo
f_goles4 = f_goles4.join(dim_players, f_goles4.player == dim_players.player, "inner")
f_goles4 = f_goles4.drop('team','player','team_id')
f_goles4 = f_goles4.withColumnRenamed('id','player_id')

# Así quedaría
f_goles4.show(5)

+--------+------+---------+
|match_id|minute|player_id|
+--------+------+---------+
|     394|     9|       30|
|     423|    55|       34|
|     562|    68|       80|
|      86|    62|       84|
|     413|    90|       85|
+--------+------+---------+
only showing top 5 rows



##### Unimos todos los goles que sacamos por evento y hacemos su tabla de hechos asignando su ID

In [19]:
# Unimos
f_goals = f_goles4.union(f_goles3).union(f_goles2).union(f_goles1)

#Asignamos ID
f_goals = f_goals.withColumn("id", monotonically_increasing_id() + 1)

# Mostrar la tabla de hechos
f_goals.show(5)

+--------+------+---------+---+
|match_id|minute|player_id| id|
+--------+------+---------+---+
|     394|     9|       30|  1|
|     423|    55|       34|  2|
|     562|    68|       80|  3|
|      86|    62|       84|  4|
|     413|    90|       85|  5|
+--------+------+---------+---+
only showing top 5 rows



#### Hecho de tarjetas rojas

##### Primero las tarjetas rojas locales

In [20]:
# Selecciono las rojas locales
f_redcard1 = df_parquet.select(
    df_parquet['home_red_card'].alias('player'),
    df_parquet['referee'].alias('referee'),
)

# Asignar un ID que es el del partido
f_redcard1 = f_redcard1.withColumn("match_id", monotonically_increasing_id() + 1)

# Limpiamos player para extraer fácil el nombre
f_redcard1 = f_redcard1.withColumn("player", regexp_replace(col("player"), r"\(P\)", ""))
f_redcard1 = f_redcard1.withColumn("player", regexp_replace(col("player"), "\s+", ""))
f_redcard1 = f_redcard1.withColumn("player", regexp_replace(col("player"), "·", ""))
f_redcard1 = f_redcard1.withColumn("player", regexp_replace(col("player"), "\|", ","))
f_redcard1 = f_redcard1.withColumn("player", regexp_replace(col("player"), r'(?<=[a-z])(?=[A-Z])', ' '))
f_redcard1 = f_redcard1.withColumn("player", regexp_replace(col("player"), r"\+.+", ""))
f_redcard1 = f_redcard1.withColumn("player", trim(col("player")))

# Hacemos un registro para cada jugador
f_redcard1 = f_redcard1.withColumn("player", explode(split(col("player"), ",")))
f_redcard1 = f_redcard1.withColumn("player", trim(col("player")))



# Quitamos el minuto del jugador y lo pasamos a la columna minute
f_redcard1 = f_redcard1.withColumn("minute", regexp_extract(col("player"), r'(\d+)$', 1))
f_redcard1 = f_redcard1.withColumn("player", regexp_extract(col("player"), r'^(.*?)\d+', 1))
f_redcard1.show(2445)

# Uno con dim team para extraer el id y quito la columna team porque ya no la ocupo
f_redcard1 = f_redcard1.join(dim_players, f_redcard1.player == dim_players.player, "inner")
f_redcard1 = f_redcard1.drop('team','player','team_id')
f_redcard1 = f_redcard1.withColumnRenamed('id','player_id')

# Uno con dim team para extraer el id y quito la columna team porque ya no la ocupo
f_redcard1 = f_redcard1.join(dim_referees, f_redcard1.referee == dim_referees.Referee, "left_outer")
f_redcard1 = f_redcard1.drop('referee')
f_redcard1 = f_redcard1.withColumnRenamed('id','referee_id')


# Nos quedaría así
f_redcard1.show(2225)

+--------------------+--------------------+--------+------+
|              player|             referee|match_id|minute|
+--------------------+--------------------+--------+------+
|         Paulo Bento|      Anthony Taylor|      34|    90|
|     Wayne Hennessey|       Mario Escobar|      45|    86|
|      Carlos Sánchez|       Damir Skomina|     112|     3|
|    Antonio Valencia|     Noumandiez Doué|     152|    50|
|   Claudio Marchisio|Marco Antonio Rod...|     154|    59|
|          Ante Rebić|     Ravshan Irmatov|     159|    89|
|           Alex Song|       Pedro Proença|     175|    40|
|        Maxi Pereira|         Felix Brych|     186|    90|
|         Luis Suárez|                NULL|     200|   120|
|      Yoann Gourcuff|                NULL|     222|    25|
|     Nicolás Lodeiro|                NULL|     256|    81|
|        Wayne Rooney|                NULL|     261|    62|
|     Marco Materazzi|                NULL|     267|    50|
|       TomášUjfaluši|                NU

##### Ahora las rojas visitantes

In [21]:
# Selecciono los rojas visitantes
f_redcard2 = df_parquet.select(
    df_parquet['away_red_card'].alias('player'),
    df_parquet['referee'].alias('referee'),
)

# Asignar un ID que es el del partido
f_redcard2 = f_redcard2.withColumn("match_id", monotonically_increasing_id() + 1)

# Limpiamos player para extraer fácil el nombre
f_redcard2 = f_redcard2.withColumn("player", regexp_replace(col("player"), r"\(P\)", ""))
f_redcard2 = f_redcard2.withColumn("player", regexp_replace(col("player"), "\s+", ""))
f_redcard2 = f_redcard2.withColumn("player", regexp_replace(col("player"), "·", ""))
f_redcard2 = f_redcard2.withColumn("player", regexp_replace(col("player"), "\|", ","))
f_redcard2 = f_redcard2.withColumn("player", regexp_replace(col("player"), r'(?<=[a-z])(?=[A-Z])', ' '))
f_redcard2 = f_redcard2.withColumn("player", regexp_replace(col("player"), r"\+.+", ""))
f_redcard2 = f_redcard2.withColumn("player", trim(col("player")))

# Hacemos un registro para cada jugador
f_redcard2 = f_redcard2.withColumn("player", explode(split(col("player"), ",")))
f_redcard2 = f_redcard2.withColumn("player", trim(col("player")))

# Quitamos el minuto del jugador y lo pasamos a la columna minute
f_redcard2 = f_redcard2.withColumn("minute", regexp_extract(col("player"), r'(\d+)$', 1))
f_redcard2 = f_redcard2.withColumn("player", regexp_extract(col("player"), r'^(.*?)\d+', 1))

# Uno con dim team para extraer el id y quito la columna team porque ya no la ocupo
f_redcard2 = f_redcard2.join(dim_players, f_redcard2.player == dim_players.player, "inner")
f_redcard2 = f_redcard2.drop('team','player','team_id')
f_redcard2 = f_redcard2.withColumnRenamed('id','player_id')

# Uno con dim team para extraer el id y quito la columna team porque ya no la ocupo
f_redcard2 = f_redcard2.join(dim_referees, f_redcard2.referee == dim_referees.Referee, "left_outer")
f_redcard2 = f_redcard2.drop('referee')
f_redcard2 = f_redcard2.withColumnRenamed('id','referee_id')


# Lo imprimimos
f_redcard2.show(2225)

+--------+------+---------+----------+
|match_id|minute|player_id|referee_id|
+--------+------+---------+----------+
|     617|    84|       75|        68|
|     693|    88|       97|       130|
|     552|    61|      101|       166|
|     770|    35|      117|         9|
|     552|    88|      149|       166|
|     729|    67|      203|       179|
|     394|    47|      267|      NULL|
|     664|    88|      286|        99|
|     831|    59|      352|       115|
|     838|    79|      383|       315|
|     514|    22|      410|        81|
|     457|    75|      458|         5|
|     817|     8|      467|       305|
|     501|    87|      488|        75|
|     929|    96|      508|       288|
|     232|    24|      514|      NULL|
|     799|    80|      536|       148|
|     772|    49|      538|        52|
|     869|    79|      547|       296|
|     772|    54|      560|        52|
|     768|    44|      566|       255|
|     935|    63|      571|       172|
|     918|    89|      60

##### Hacemos la tabla de hechos

In [22]:
# Unimos
f_red_cards = f_redcard1.union(f_redcard2)

#Asignamos ID
f_red_cards = f_red_cards.withColumn("id", monotonically_increasing_id() + 1)

# Mostrar la tabla de hechos
f_red_cards.show(2555)

+--------+------+---------+----------+----------+
|match_id|minute|player_id|referee_id|        id|
+--------+------+---------+----------+----------+
|     175|    40|       54|        32|         1|
|     427|    68|       63|      NULL|         2|
|     159|    89|      109|        64|         3|
|     152|    50|      118|        80|         4|
|     918|    89|      132|        36|         5|
|     374|    59|      171|      NULL|         6|
|     112|     3|      207|       155|         7|
|     390|    40|      225|      NULL|         8|
|     154|    59|      229|        20|         9|
|     416|    53|      238|      NULL|        10|
|     297|    28|      261|      NULL|        11|
|     613|    85|      293|       262|        12|
|     546|    52|      351|       190|        13|
|     570|    88|      409|       320|        14|
|     799|    83|      431|       148|        15|
|     423|    54|      453|      NULL|        16|
|     484|    21|      459|       159|        17|


## Load

#### En mi caso los pasaré a csv para que todos puedan acceder a ellos por si quisieran usarlos

In [23]:
# dim_teams
dim_teams_pandas = dim_teams.toPandas()
dim_teams_pandas.to_csv('dim_teams.csv', index=False, encoding='utf-8')

# dim_referees
dim_referees_pandas = dim_referees.toPandas()
dim_referees_pandas.to_csv('dim_referees.csv', index=False, encoding='utf-8')

# dim_matches
dim_matches_pandas = dim_matches.toPandas()
dim_matches_pandas.to_csv('dim_matches.csv', index=False, encoding='utf-8')

# dim_players
dim_players_pandas = dim_players.toPandas()
dim_players_pandas.to_csv('dim_players.csv', index=False, encoding='utf-8')

# f_goals
f_goals_pandas = f_goals.toPandas()
f_goals_pandas.to_csv('f_goals.csv', index=False, encoding='utf-8')

# f_red_cards
f_red_cards_pandas = f_red_cards.toPandas()
f_red_cards_pandas.to_csv('f_red_cards.csv', index=False, encoding='utf-8')

# Descargar cada archivo CSV
files.download('dim_teams.csv')
files.download('dim_referees.csv')
files.download('dim_matches.csv')
files.download('dim_players.csv')
files.download('f_goals.csv')
files.download('f_red_cards.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>